In [1]:
suppressMessages(suppressWarnings(source("../config/config_sing.R")))
show_env()

You are in Singularity: singularity_proj_combeffect 
BASE DIRECTORY:     /mount/work 
PATH OF SOURCE:     /mount/work/source 
PATH OF EXECUTABLE: /mount/work/exe 
PATH OF ANNOTATION: /mount/work/annotation 
PATH OF PROJECT:    /mount/project 
PATH OF RESULTS:    /mount/work/out/proj_combeffect_encode_fcc 


## Import data

In [2]:
PREFIX = "A001_K562_WSTARRseq"
FOLDER = "coverage_astarrseq_peak_macs_input"

In [8]:
fdiry = file.path(FD_RES, PREFIX, FOLDER, "summary")

fname = "wgs_count_matrix.tsv"
fpath = file.path(fdiry, fname)
dat_count = read_tsv(fpath)

fname = "wgs_metadata.tsv"
fpath = file.path(fdiry, fname)
dat_meta = read_tsv(fpath)

fdiry = file.path(FD_RES, PREFIX, "coverage", "library_size")
fname = "library_size_summary.csv"
fpath = file.path(fdiry, fname)
dat_lib = read_csv(fpath) %>% 
    dplyr::mutate(Size = Count) %>% 
    dplyr::select(Sample, Group, Size)

Rows: 246832 Columns: 11
── Column specification ──────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
Delimiter: "\t"
chr (2): Chrom, Peak
dbl (9): Start, End, Input_rep1, Input_rep2, Input_rep3, Input_rep4, Output_...

ℹ Use `spec()` to retrieve the full column specification for this data.
ℹ Specify the column types or set `show_col_types = FALSE` to quiet this message.
Rows: 7 Columns: 3
── Column specification ──────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
Delimiter: "\t"
chr (3): Sample, Group, FPath

ℹ Use `spec()` to retrieve the full column specification for this data.
ℹ Specify the column types or set `show_col_types = FALSE` to quiet this message.
Rows: 7 Columns: 5
── Column specification ──────────────────────────────────────────────────────────

In [9]:
head(dat_count)

Chrom,Start,End,Peak,Input_rep1,Input_rep2,Input_rep3,Input_rep4,Output_rep1,Output_rep2,Output_rep3
<chr>,<dbl>,<dbl>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
chr1,10015,10442,chr1_10015_10442,1,1,1,1,NA,NA,NA
chr1,17237,17772,chr1_17237_17772,5,12,15,17,18,23,27
chr1,136071,137429,chr1_136071_137429,3,4,7,4,12,12,29
chr1,137737,139544,chr1_137737_139544,14,40,41,52,145,144,217
chr1,180982,182087,chr1_180982_182087,8,31,26,28,63,57,99
chr1,183239,184602,chr1_183239_184602,12,40,36,49,71,75,163


In [10]:
dat_meta

Sample,Group,FPath
<chr>,<chr>,<chr>
Input_rep1,Input,/mount/work/out/proj_combeffect_encode_fcc/A001_K562_WSTARRseq/coverage_astarrseq_peak_macs_input/A001-input-K562-rep1.masked.dedup.fragments.counts.txt.gz
Input_rep2,Input,/mount/work/out/proj_combeffect_encode_fcc/A001_K562_WSTARRseq/coverage_astarrseq_peak_macs_input/A001-input-K562-rep2.masked.dedup.fragments.counts.txt.gz
Input_rep3,Input,/mount/work/out/proj_combeffect_encode_fcc/A001_K562_WSTARRseq/coverage_astarrseq_peak_macs_input/A001-input-K562-rep3.masked.dedup.fragments.counts.txt.gz
Input_rep4,Input,/mount/work/out/proj_combeffect_encode_fcc/A001_K562_WSTARRseq/coverage_astarrseq_peak_macs_input/A001-input-K562-rep4.masked.dedup.fragments.counts.txt.gz
Output_rep1,Output,/mount/work/out/proj_combeffect_encode_fcc/A001_K562_WSTARRseq/coverage_astarrseq_peak_macs_input/A001-K562-rep1.masked.dedup.fragments.counts.txt.gz
Output_rep2,Output,/mount/work/out/proj_combeffect_encode_fcc/A001_K562_WSTARRseq/coverage_astarrseq_peak_macs_input/A001-K562-rep2.masked.dedup.fragments.counts.txt.gz
Output_rep3,Output,/mount/work/out/proj_combeffect_encode_fcc/A001_K562_WSTARRseq/coverage_astarrseq_peak_macs_input/A001-K562-rep3.masked.dedup.fragments.counts.txt.gz


In [11]:
dat_lib

Sample,Group,Size
<chr>,<chr>,<dbl>
Input_rep1,Input,26908970
Input_rep2,Input,99899775
Input_rep3,Input,105623984
Input_rep4,Input,108635002
Output_rep1,Output,160349140
Output_rep2,Output,157326312
Output_rep3,Output,328185275


## Calculate CPM for each sample

In [12]:
mat = dat_count %>% dplyr::select(-Chrom, -Start, -End, -Peak)
all(colnames(mat) == dat_lib$Sample)

[1] TRUE

In [13]:
### init
size  = dat_lib$Size
group = dat_lib$Group
peak  = dat_count$Peak

### CPM
mat = dat_count %>% dplyr::select(-Chrom, -Start, -End, -Peak)
mat = apply(mat, 1, function(x){
    y = x * 10^6 / size
    y = tapply(y, group, mean)
    return(y)
})

### arrange
mat = t(mat)
rownames(mat) = peak
dat_cpm = as.data.frame(mat) %>% rownames_to_column(var = "Peak")
head(dat_cpm)

,Peak,Input,Output
,<chr>,<dbl>,<dbl>
1,chr1_10015_10442,0.01646126,NA
2,chr1_17237_17772,0.15110814,0.11357287
3,chr1_136071_137429,0.06365512,0.07982534
4,chr1_137737_139544,0.44687760,0.82692790
5,chr1_180982_182087,0.27787742,0.35228528
6,chr1_183239_184602,0.40955815,0.47205688


## Save the table

In [14]:
fdiry = file.path(FD_RES, PREFIX, FOLDER, "summary")
fname = "wgs_summary_cpm.tsv"
fpath = file.path(fdiry, fname)
print(fpath)

write_tsv(dat_cpm, fpath)

[1] "/mount/work/out/proj_combeffect_encode_fcc/A001_K562_WSTARRseq/coverage_astarrseq_peak_macs_input/summary/wgs_summary_cpm.tsv"
